In [1]:
import os 
import sys
sys.path.append('../')
import torchvision
from gaussian_teleport.edm.dataset import ImageFolderDataset
from gaussian_teleport.edm.networks_edm import SongUNet
from gaussian_teleport.edm.train_edm import create_model, EDM, train_edm_model
from gaussian_teleport.edm_utils import get_default_config

In [2]:
config = get_default_config("mnist")
unet = create_model(config)
edm = EDM(model=unet, cfg=config)
edm.model.train();

In [3]:
config

{'DATASET': 'mnist',
 'channel_mult': [1, 2, 3, 4],
 'model_channels': 16,
 'attn_resolutions': [0],
 'layers_per_block': 1,
 'num_fid_sample': 5000,
 'fid_batch_size': 1024,
 'channels': 1,
 'img_size': 32,
 'device': 'cuda',
 'sigma_min': 0.002,
 'sigma_max': 80.0,
 'rho': 7.0,
 'sigma_data': 0.5,
 'spatial_matching': 'padding'}

In [ ]:
img_dataset = torchvision.datasets.MNIST(root='datasets/mnist', download=True, train=True,
                                    transform=torchvision.transforms.Compose(
                                        [torchvision.transforms.Resize(config.img_size),
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5,), (0.5,))]
                                    ),)

In [ ]:
outdir = f"exps/MNIST_temp"
sample_dir = f"{outdir}/samples"
ckpt_dir = f"{outdir}/checkpoints"
config.outdir = outdir
config.sample_dir = sample_dir
config.ckpt_dir = ckpt_dir
for dir in [outdir, sample_dir, ckpt_dir]:
    os.makedirs(dir, exist_ok=True)

config.train_batch_size = 512
config.num_steps = 20000
config.learning_rate = 2e-4
config.train_progress_bar = True
config.total_steps = 18
config.save_images_step = 100
config.eval_batch_size = 64
config.accumulation_steps = 1
config.gt_guide_type = 'l2'
config.warmup = 1000
config.log_step = 100
config.save_model_iters = 5000

# around 1000 steps you could see some digits. 
train_edm_model(config, unet, img_dataset, device="cpu")